
# Machine Learning Project Proposal #

## Exploring Used Car Auction: *Prices and Predicting the Possible End Price*

### Simon Coessens, Rana Islek

### April 2024



This code was executed in Kaggle because there the hardware available was more suited.

### Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import skew

In [ ]:
pd.set_option('display.max_columns', None)
%matplotlib inline
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.set_style("whitegrid")
sns.color_palette("dark")
plt.style.use("fivethirtyeight")

### Import Data

In [ ]:
dfcar = pd.read_csv('/kaggle/input/dddddd/dfcar_processed.csv', on_bad_lines='skip')
dfcar.head(2)

,vin,year,brand,body_simple,model,trim,made_in,transmission,condition,color,interior,car_age,odometer,state,seller,sellingprice,mmr,is_sold_below_mmr,day_of_week,is_weekend,hour_only,month_year,sale_date,sale_year,latitude,longitude
0,wba3c1c51ek116351,2014,BMW,Sedan,3 Series,328I SULEV,DEU,Automatic,4.5,Gray,Black,1,1331.0,CA,Financial Services Remarketing (Lease),30000,31900,1,Thu,0,4,Jan-15,2015-01-15,2015,37.271875,-119.270415
1,yv1612tb4f1310987,2015,VOLVO,Sedan,S60,T5,SWE,Automatic,4.1,White,Black,0,14282.0,CA,Volvo Na Rep/World Omni,27750,27500,0,Thu,0,4,Jan-15,2015-01-29,2015,37.271875,-119.270415


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [ ]:
columns_to_drop = ['year','vin','mmr', 'is_sold_below_mmr', 'day_of_week', 'is_weekend', 'is_weekend', 'month_year', 'sale_date', 'latitude', 'longitude', 'hour_only', 'seller']
dfcar = dfcar.drop(columns=columns_to_drop)
dfcar.head(2)

,brand,body_simple,model,trim,made_in,transmission,condition,color,interior,car_age,odometer,state,sellingprice,sale_year
0,BMW,Sedan,3 Series,328I SULEV,DEU,Automatic,4.5,Gray,Black,1,1331.0,CA,30000,2015
1,VOLVO,Sedan,S60,T5,SWE,Automatic,4.1,White,Black,0,14282.0,CA,27750,2015


In [ ]:
categorical_features = ['brand', 'body_simple', 'model', 'trim', 'made_in', 'transmission', 'color', 'state', 'interior']
numerical_features = ['condition', 'odometer', 'car_age', 'car_age', 'sale_year']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])


In [ ]:
X = dfcar.drop('sellingprice', axis=1)
y = dfcar['sellingprice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Pipeline([
    ('preprocessor', preprocessor),
    ('svr', SVR(kernel='linear'))
])

model.fit(X_train, y_train)

In [ ]:
from joblib import dump


model_filename = 'svm_model.joblib'
dump(model, model_filename)
print(f"Model saved to {model_filename}")

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")